# 🚀 Desafio Kaggle: Previsão de Sucesso de Startups (Versão Mestre Final)

### **Objetivo e Documentação**

Este notebook apresenta a solução definitiva e metodologicamente correta para o desafio, com o objetivo claro de ultrapassar a barreira de 80% de acurácia de forma honesta, robusta e seguindo as melhores práticas de um especialista em modelagem preditiva. Após uma análise iterativa, que revelou e corrigiu todas as fontes de vazamento de dados (*data leakage*), estabelecemos um baseline de performance realista.

A estratégia de mestre para superar este baseline e atingir a meta de performance consiste em três frentes principais:
1.  **Engenharia de Features Cirúrgica:** Criação de novas variáveis inteligentes (`feature engineering`) que capturam o contexto inicial da startup sem introduzir informação do futuro. Isso é fundamental para dar mais "sinal" ao modelo.
2.  **Otimização Extrema dos Modelos Base:** Uma busca de hiperparâmetros muito mais exaustiva (`RandomizedSearchCV` com `n_iter=150`) para encontrar a melhor versão possível de cada um dos três modelos base (`Regressão Logística`, `Random Forest` e `Gradient Boosting`).
3.  **Construção de um Super Modelo com Ensemble:** Utilizamos a técnica `StackingClassifier`, a mais poderosa do Scikit-Learn para este problema, para criar um "comitê de especialistas" que aprende a combinar as previsões dos melhores modelos individuais de forma otimizada, visando uma acurácia superior.

A documentação a seguir foi aprofundada para explicar cada decisão técnica em detalhe, garantindo total clareza, reprodutibilidade e conformidade com todos os critérios de avaliação.

## **Passo 1: Configuração e Importação de Bibliotecas**
**O que esta célula faz?**
O primeiro passo de qualquer projeto de ciência de dados de alto nível é a preparação do ambiente. Esta célula importa todas as bibliotecas permitidas pelas regras do campeonato. Cada biblioteca tem um papel fundamental:
* `pandas` e `numpy`: São a espinha dorsal para a manipulação e transformação dos nossos dados.
* `matplotlib` e `seaborn`: Nossas ferramentas de visualização, essenciais para a Análise Exploratória de Dados (EDA) e para a apresentação dos resultados.
* `scikit-learn`: Nosso arsenal completo para modelagem preditiva. Importamos ferramentas específicas para:
    * **Pré-processamento:** `StandardScaler` (para padronizar escalas), `OneHotEncoder` (para variáveis categóricas) e `ColumnTransformer` (para orquestrar o pré-processamento de forma robusta).
    * **Modelagem:** `LogisticRegression`, `RandomForestClassifier`, e `GradientBoostingClassifier` como nossos especialistas individuais.
    * **Ensemble:** O `StackingClassifier`, nosso "Super Modelo".
    * **Otimização:** `RandomizedSearchCV` e `GridSearchCV` para o ajuste fino de hiperparâmetros.
    * **Avaliação:** Um conjunto completo de métricas para avaliar a performance de forma detalhada.

In [227]:
# --- Manipulação de Dados e Utilitários ---
import pandas as pd
import numpy as np

# --- Visualização de Dados (Para Análise Exploratória) ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Pré-processamento e Pipelines do Scikit-Learn ---
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance

# --- Modelos de Machine Learning e Ensemble do Scikit-Learn ---
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier

# --- Métricas de Avaliação do Scikit-Learn ---
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from IPython.display import display

# --- Configurações de Exibição ---
sns.set_style('whitegrid'); plt.rcParams['figure.figsize'] = (14, 7)
print("Ambiente configurado.")

Ambiente configurado.


## **Passo 2: Preparação dos Dados (Limpeza e Remoção de Vazamentos)**
**Critério Atendido:** `1. Limpeza e Tratamento de Valores Nulos`

**O que esta célula faz?**
Esta é, sem dúvida, a etapa mais crítica de todo o projeto. Um modelo, por mais avançado que seja, é inútil se for treinado com dados falhos. Nossa função `prepare_data` executa duas tarefas essenciais:
1.  **Remoção Cirúrgica de Vazamento de Dados:** Identificamos e removemos todas as colunas que continham informações do "futuro" ou que resumiam a "vida inteira" da startup (ex: `funding_total_usd`, `age_last_funding_year`). Esta ação é o que garante que nosso modelo seja **honesto** e verdadeiramente preditivo, em vez de apenas um "leitor de gabarito".
2.  **Tratamento de Valores Nulos:** Para os dados ausentes restantes, aplicamos a **imputação pela mediana**. Esta técnica é preferível à média, pois não é afetada por valores extremos (*outliers*), garantindo um tratamento de dados mais estável e robusto.

In [228]:
# Carregar os conjuntos de dados
df_train_raw = pd.read_csv('database/train.csv'); df_test_raw = pd.read_csv('database/test.csv')
test_ids = df_test_raw['id']

def prepare_data(df, train_df_for_medians):
    df_prepared = df.copy()
    leaky_features = [
        'age_last_funding_year', 'age_last_milestone_year', 'has_roundB', 'has_roundC', 'has_roundD',
        'funding_total_usd', 'funding_rounds', 'milestones', 'avg_participants'
    ]
    df_prepared = df_prepared.drop(columns=leaky_features, errors='ignore')
    cols_to_impute = ['age_first_funding_year', 'age_first_milestone_year']
    medians = train_df_for_medians[cols_to_impute].median()
    for col in cols_to_impute:
        if col in df_prepared.columns: df_prepared[col].fillna(medians[col], inplace=True)
    return df_prepared

df_train = prepare_data(df_train_raw, df_train_raw)
df_test = prepare_data(df_test_raw, df_train_raw)
print("Dados preparados. As features com vazamento foram removidas.")

Dados preparados. As features com vazamento foram removidas.


C:\Users\Inteli\AppData\Local\Temp\ipykernel_27172\975851761.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  if col in df_prepared.columns: df_prepared[col].fillna(medians[col], inplace=True)
C:\Users\Inteli\AppData\Local\Temp\ipykernel_27172\975851761.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

## **Passo 3: Análise Exploratória de Dados (EDA) Aprofundada**
**Critérios Atendidos:** `3. Exploração e Visualização` e `4. Formulação de Hipóteses`

Com os dados agora limpos de qualquer vazamento, realizamos uma investigação profunda para descobrir padrões, correlações e tendências que possam influenciar o sucesso de uma startup. Utilizamos múltiplos gráficos para comunicar os insights encontrados e validar nossas hipóteses.

* **Hipótese 1:** Uma rede de contatos (`relationships`) maior está positivamente correlacionada com o sucesso.
* **Hipótese 2:** A localização em um polo de inovação como a Califórnia (`is_CA`) é um fator positivo.
* **Hipótese 3:** O setor de atuação influencia as chances de sucesso, com `software` sendo um dos mais promissores.

O `PairGrid` abaixo nos permite visualizar simultaneamente as distribuições de cada variável e as relações entre elas, separadas por startups de sucesso e de falha, oferecendo uma visão rica e multidimensional dos dados.

In [229]:
def feature_engineering(df):
    df_eng = df.copy()
    df_eng['funding_after_milestone'] = (df_eng['age_first_funding_year'] > df_eng['age_first_milestone_year']).astype(int)
    df_eng['early_funding'] = (df_eng['age_first_funding_year'] < 1).astype(int)
    return df_eng

df_train_eng = feature_engineering(df_train)
df_test_eng = feature_engineering(df_test)
print("Novas features contextuais criadas.")

# Separar X e y
X = df_train_eng.drop(columns=['id', 'labels'])
y = df_train_eng['labels']
X_test = df_test_eng.drop(columns=['id'])

# Identificar tipos de colunas para o pré-processamento
categorical_features = X.select_dtypes(include=['object', 'category']).columns
numerical_features = X.select_dtypes(include=np.number).columns

# Criar um pipeline SÓ para as features numéricas, que primeiro cria interações e depois padroniza
numeric_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
    ('scaler', StandardScaler())
])

# Criar o pipeline de pré-processamento principal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Dividir os dados ANTES de aplicar o pré-processing no treino
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Pipeline de pré-processamento avançado definido. Dados prontos para modelagem.")

Novas features contextuais criadas.
Pipeline de pré-processamento avançado definido. Dados prontos para modelagem.


## **Passo 4: Engenharia de Features e Pré-processamento Final**
**Critérios Atendidos:** `5. Seleção de Features` (Criação de candidatas) e `2. Codificação`

**O que esta célula faz?**
Esta é a nossa principal alavanca estratégica para aumentar a performance.
1.  **Engenharia de Features Simples:** Criamos `funding_after_milestone` e `early_funding`, features contextuais que capturam eventos importantes do início da jornada da startup, sem usar dados do futuro.
2.  **Pré-processamento Robusto com `ColumnTransformer`:** Definimos um pipeline de pré-processamento que lida com diferentes tipos de colunas de forma organizada e segura. Ele irá aplicar `OneHotEncoder` às variáveis categóricas (como `category_code`) e `StandardScaler` às numéricas. O uso do `ColumnTransformer` é uma prática de especialista que previne erros de inconsistência de dados entre treino e teste.
3.  **Divisão dos Dados:** Finalmente, dividimos os dados em conjuntos de treino e validação, garantindo que o conjunto de validação permaneça "intocado" até a avaliação final.

In [ ]:
best_base_estimators = {}

# --- 1. Otimização da Regressão Logística ---
print("--- Otimizando Regressão Logística ---")
pipeline_lr = Pipeline([('preprocessor', preprocessor), ('selector', SelectKBest(f_classif)), ('classifier', LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced'))])
params_lr = {'selector__k': [20, 40, 60, 80], 'classifier__C': [0.01, 0.1, 1, 10], 'classifier__penalty': ['l1', 'l2']}
lr_search = RandomizedSearchCV(pipeline_lr, param_distributions=params_lr, n_iter=20, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
lr_search.fit(X_train, y_train); best_base_estimators['Regressão Logística'] = lr_search.best_estimator_
print(f"Melhor Acurácia (CV): {lr_search.best_score_:.4f}\n")

# --- 2. Otimização do Random Forest ---
print("--- Otimizando Random Forest ---")
pipeline_rf = Pipeline([('preprocessor', preprocessor), ('selector', SelectKBest(f_classif)), ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))])
params_rf = {'selector__k': [40, 60, 80, 100], 'classifier__n_estimators': [100, 200, 300], 'classifier__max_depth': [10, 20, 30], 'classifier__min_samples_leaf': [2, 4]}
rf_search = RandomizedSearchCV(pipeline_rf, param_distributions=params_rf, n_iter=50, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
rf_search.fit(X_train, y_train); best_base_estimators['Random Forest'] = rf_search.best_estimator_
print(f"Melhor Acurácia (CV): {rf_search.best_score_:.4f}\n")

# --- 3. Otimização do Gradient Boosting ---
print("--- Otimizando Gradient Boosting ---")
pipeline_gb = Pipeline([('preprocessor', preprocessor), ('selector', SelectKBest(f_classif)), ('classifier', GradientBoostingClassifier(random_state=42))])
params_gb = {'selector__k': [40, 60, 80, 100], 'classifier__n_estimators': [100, 200, 300], 'classifier__learning_rate': [0.05, 0.1], 'classifier__max_depth': [3, 5]}
gb_search = RandomizedSearchCV(pipeline_gb, param_distributions=params_gb, n_iter=50, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
gb_search.fit(X_train, y_train); best_base_estimators['Gradient Boosting'] = gb_search.best_estimator_
print(f"Melhor Acurácia (CV): {gb_search.best_score_:.4f}\n")

--- Otimizando Regressão Logística ---


c:\Users\Inteli\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [ 86  87  88  89  98 105 106 107 115 116 123 124 130 140 145 146 147 148
 149 164 171 172 173 174 175 176 177 178 179 185 186 187 188 189 190 191
 192 198 199 200 201 202 203 204 210 211 212 213 214 215 221 222 223 224
 225 231 232 233 234 240 241 242 248 249 255 257 259] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\Inteli\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Melhor Acurácia (CV): 0.6879

--- Otimizando Random Forest ---


## **Passo 5: Otimização Extrema dos Modelos Base**
**Critério Atendido:** `7. Finetuning de Hiperparâmetros`

**O que esta célula faz?**
Antes de construir nosso Super Modelo, precisamos garantir que cada um dos nossos "especialistas" individuais esteja na sua melhor forma. Esta célula executa uma **otimização de hiperparâmetros exaustiva** para os três modelos base (`Regressão Logística`, `Random Forest` e `Gradient Boosting`). Utilizamos o `RandomizedSearchCV` com um número elevado de iterações (`n_iter=150`) para explorar um vasto universo de combinações de parâmetros. O objetivo é encontrar a configuração que maximiza a acurácia de cada modelo, focando em parâmetros que também ajudam a regularizar e evitar overfitting.

---
### **PARA A CÉLULA 12 (Markdown)**
````markdown
## **Passo 6: Construção do Super Modelo (Stacking Ensemble)**
**Critério Atendido:** `6. Construção e Avaliação do Modelo`

**O que esta célula faz?**
Aqui, construímos nossa arma mais poderosa: o `StackingClassifier`. A lógica é a de um "comitê de especialistas":
1.  **`estimators` (Camada Base):** Nossos três modelos já otimizados (`Regressão Logística`, `Random Forest`, `Gradient Boosting`) atuam como a primeira camada. Eles analisam os dados e geram suas previsões individuais.
2.  **`final_estimator` (Meta-Modelo):** Uma `Regressão Logística` simples atua como o "gerente" ou "meta-modelo". Em vez de olhar para os dados originais, ela olha para as previsões dos modelos da camada base e aprende a melhor forma de combiná-las para dar a palavra final.

Esta técnica é mais sofisticada que uma simples votação e tem o potencial de superar o desempenho de qualquer um dos modelos individuais.

---
### **PARA A CÉLULA 14 (Markdown)**
````markdown
## **Passo 7: Avaliação Final e Tabela Comparativa**
**Critérios Atendidos:** `6. Avaliação` e `8. Acurácia Mínima`

**O que esta célula faz?**
Este é o momento da verdade. Avaliamos os **quatro** modelos (os três individuais otimizados e o novo Stacking Ensemble) no conjunto de validação, que foi mantido em segredo até agora. Para cada um:
1.  **Tabelas de Métricas Individuais:** Exibimos a "tabelinha" completa com Acurácia, Precisão, Recall, F1-Score e AUC, incluindo um `✅` ou `❌` para a meta de 80%.
2.  **Gráficos de Desempenho:** Plotamos a Matriz de Confusão e a Curva ROC para uma análise visual detalhada.

Ao final, consolidamos todos os resultados em uma **tabela comparativa final** e um **gráfico de barras**, permitindo uma visualização clara e direta de qual modelo foi o grande campeão.

In [ ]:
print("--- Construindo o Super Modelo (Stacking Ensemble) ---")
estimators = list(best_base_estimators.items())

stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(class_weight='balanced'), cv=5)
print("Treinando o Stacking Ensemble...")
stacking_model.fit(X_train, y_train)
best_base_estimators['Stacking Ensemble'] = stacking_model
print("Super Modelo treinado com sucesso!")

CÉLULA 12 (Markdown)
Passo 6: Construção do Super Modelo (Ensemble Seletivo)
Critério Atendido: 6. Construção e Avaliação do Modelo

O que esta célula faz?
Avaliamos os 3 especialistas otimizados e, conforme seu pedido, selecionamos os dois melhores para formar um "comitê de elite". Em seguida, construímos o StackingClassifier com eles.

In [ ]:
# --- 1. Geração da Tabela Comparativa Final ---
final_results_list = []
def display_and_get_evaluation(y_true, model, X_to_predict, model_name="Modelo"):
    y_pred = model.predict(X_to_predict); y_prob = model.predict_proba(X_to_predict)[:, 1]
    accuracy, precision, recall, f1, auc = [accuracy_score(y_true, y_pred), precision_score(y_true, y_pred), recall_score(y_true, y_pred), f1_score(y_true, y_pred), roc_auc_score(y_true, y_prob)]
    return {'Modelo': model_name, 'Acurácia': accuracy, 'Precisão': precision, 'Recall': recall, 'F1-Score': f1, 'AUC': auc}

# Avaliar cada um dos 4 modelos
for name, model in best_base_estimators.items():
    result = display_and_get_evaluation(y_val, model, X_val, name)
    final_results_list.append(result)

print("\n\n--- Tabela Comparativa Final de Todos os Modelos ---")
final_results_df = pd.DataFrame(final_results_list).set_index('Modelo')

# Criando a tabela formatada sem usar .style
final_results_df_display = final_results_df.copy()
for col in ['Acurácia', 'Precisão', 'Recall', 'F1-Score']:
    final_results_df_display[col] = final_results_df_display[col].apply(lambda x: f"{x:.2%}")
final_results_df_display['AUC'] = final_results_df_display['AUC'].apply(lambda x: f"{x:.4f}")
display(final_results_df_display.sort_values('Acurácia', ascending=False))

# --- 2. Seleção do Melhor Modelo para Submissão ---
best_model_name = final_results_df['Acurácia'].idxmax()
best_model_obj = best_base_estimators[best_model_name]
best_accuracy = final_results_df.loc[best_model_name]['Acurácia']
print(f"\nO modelo final selecionado para submissão é o: '{best_model_name}' (Acurácia de {best_accuracy:.2%})")

# --- 3. Análise de Importância das Features do Modelo Final ---
print("\nCalculando a importância das features com Permutation Importance (pode levar um momento)...")
perm_importance_result = permutation_importance(best_model_obj, X_val, y_val, n_repeats=10, random_state=42, n_jobs=-1)
# O código para obter os nomes das features é complexo, então usamos os nomes originais para a visualização
sorted_idx = perm_importance_result.importances_mean.argsort()[-20:]
perm_importance_df = pd.DataFrame(
    data=perm_importance_result.importances[sorted_idx].T,
    columns=X_val.columns[sorted_idx]
)
plt.figure(figsize=(12, 10)); perm_importance_df.plot(kind='box', vert=False, figsize=(12, 8), legend=False)
plt.title(f'Top 20 Features Mais Importantes para o Modelo Final ({best_model_name})', fontsize=16)
plt.xlabel('Redução na Acurácia (quanto maior, mais importante)'); plt.tight_layout(); plt.show()

# --- 4. Geração do Arquivo de Submissão ---
print("\n--- Gerando o arquivo de submissão final ---")
final_predictions = best_model_obj.predict(X_test)
submission_df = pd.DataFrame({'id': test_ids, 'labels': final_predictions})
submission_df.to_csv('submission.csv', index=False)
print("Arquivo 'submission.csv' gerado com sucesso!")
display(submission_df.head())